<a href="https://colab.research.google.com/github/kwankao26/229352-Course-Name-Statistical-Learning-for-Data-Science-2-Semester-1-2025/blob/Lab/660510753_Lab04_Naive_Bayes_Grid_and_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [ ]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB(alpha = 0.1)

### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

pipeline = Pipeline([('count', CountVectorizer(stop_words='english')),
                     ('nb', MultinomialNB())])

parameters = {'nb__alpha': uniform(loc = 0.1, scale = 10)}

clf = RandomizedSearchCV(pipeline, parameters, n_iter = 10, cv = 3) #n__iter=10คือการสุ่ม 10 ครั้ง #cv=3คือการแบ่งtrainningsetเป็น3ส่วน
clf.fit(Xtrain, ytrain)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('count',
                                              CountVectorizer(stop_words='english')),
                                             ('nb', MultinomialNB())]),
                   param_distributions={'nb__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ff3f078be10>})

#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [ ]:
#1
from sklearn.metrics import f1_score
param_grid = {'nb__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}

grid_clf = GridSearchCV(pipeline, param_grid, cv=5)
grid_clf.fit(Xtrain, ytrain)

print("Best alpha (grid):", grid_clf.best_params_['nb__alpha'])
y_pred_grid = grid_clf.predict(Xtest)

Best alpha (grid): 0.5


In [ ]:
#2
best_alpha_grid = grid_clf.best_params_['nb__alpha']
y_pred_grid = grid_clf.predict(Xtest)

f1_macro_grid = f1_score(ytest, y_pred_grid, average='macro')

print("Best alpha (from Grid Search):", best_alpha_grid)
print("f1_macro score on test set: {:.3f}".format(f1_macro_grid))

Best alpha (from Grid Search): 0.5
f1_macro score on test set: 0.983


In [ ]:
#3
param_dist = {'nb__alpha': uniform(loc=0.001, scale=10)}

random_clf = RandomizedSearchCV(pipeline, param_dist, n_iter=10, cv=5)

random_clf.fit(Xtrain, ytrain)

best_alpha_random = random_clf.best_params_['nb__alpha']

y_pred_random = random_clf.predict(Xtest)

f1_macro_random = f1_score(ytest, y_pred_random, average='macro')

print("Best alpha (Randomized Search): {:.3f}".format(best_alpha_random))
print("f1_macro score (Randomized Search): {:.3f}".format(f1_macro_random))

# เปรียบเทียบกับ Grid Search (ข้อ 2)
if f1_macro_random > f1_macro_grid:
    print("โมเดล Randomized Search ดีกว่า Grid Search")
elif f1_macro_random == f1_macro_grid:
    print("คะแนนของโมเดล Randomized Search และ Grid Search เท่ากัน")
else:
    print("โมเดล Grid Search ดีกว่า Randomized Search")

Best alpha (Randomized Search): 2.258
f1_macro score (Randomized Search): 0.983
โมเดล Grid Search ดีกว่า Randomized Search
